In [7]:
!pip install -qU langchain tiktoken gpt4all streamlit-chat einops transformers accelerate 

## Import all dependencies

In [8]:
from langchain import ConversationChain,PromptTemplate
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import  VectorstoreIndexCreator
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.llms import GPT4All

from langchain.memory import VectorStoreRetrieverMemory

import streamlit as st
from streamlit_chat import message


Create `loader` which loads the files from the obsidian vault. After that we run `loader.load()` to just check how many files were loaded. 

In [9]:
loader = DirectoryLoader('D:/OneDrive/Documents/Obsidian/Projects/myVault/', glob="**/*.md",recursive=True, show_progress=True, use_multithreading=True, loader_cls=TextLoader)
docs = loader.load()
len(docs)

100%|██████████| 35/35 [00:00<00:00, 4375.45it/s]


35

Create `embeddings` using the Hugging Face hosted `sentence-transformers/all-mpnet-base-v2`

In [10]:
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
llm = GPT4All(model="./ggml-mpt-7b-instruct.bin", top_p=0.5, top_k=0,  temp=0.5, repeat_penalty=1.1, n_threads=12, n_batch=16)

Found model file.


Create a vector database using the `VectorStoreIndexCreator`.

In [11]:
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])
retriever = index.vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever)


100%|██████████| 35/35 [00:00<00:00, 4997.98it/s]


### MPT-7B Instruct model was trained on data formatted in the dolly-15k format like shown:

In [13]:
_DEFAULT_TEMPLATE = """
Below is an instruction that describes a task. Write a response that appropriately completes the request.
###Instruction: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Do not make up answers and provide only information that you have.
Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)
{input}

### Response:
"""

In [14]:
PROMPT = PromptTemplate(
    input_variables=[ "history", "input"], template=_DEFAULT_TEMPLATE
)

conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory = memory,
    verbose=True
    )


with torch.inference_mode():

   conversation_with_summary.predict(input = "give me more details")




> Entering new ConversationChain chain...
Prompt after formatting:

Below is an instruction that describes a task. Write a response that appropriately completes the request.
###Instruction: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Do not make up answers and provide only information that you have.
Relevant pieces of previous conversation:
![https://res.cloudinary.com/dltwftrgc/image/upload/v1683657527/Blogs/How%20to%20Build%20an%20AI-Powered%20Game%20Bot%20with%20PyTorch%20and%20EfficientNet/efficientnet_xqnfot.png](https://res.cloudinary.com/dltwftrgc/image/upload/v1683657527/Blogs/AI_powered_game_bot/efficientnet_xqnfot.png)

```python
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torchvision.models.efficientnet_v2_s()
model.classifier = torch.nn.Linear(in_features = 1280, out_f